In [1]:
from azureml.core import Dataset
from azureml.core import Workspace
import datetime

ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='BTC-Bitstamp-raw')

df = dataset.to_pandas_dataframe()
df = df.rename(columns={"Timestamp": "dateid", "Close": "price","Volume_(BTC)":"volume"})


df

,dateid,Open,High,Low,price,volume,Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [2]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
df['date'] = [dateparse(x) for x in df['dateid']]
df = df.set_index('date')
del df['dateid']
df['ask'] = df['price']*1.01
df['bid'] = df['price']*0.99
df = df.reindex(columns=['price','ask','bid','volume','High','Low'])
df

,price,ask,bid,volume,High,Low
date,,,,,,
2011-12-31 07:52:00,4.39,4.4339,4.3461,0.455581,4.39,4.39
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,58714.31,58686.00
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,58693.43,58683.97
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,58723.84,58693.43


In [3]:
#df = df.dropna()

def delta(timeseriesShort, timeseriesLong):
    return ((timeseriesShort/timeseriesLong)-1).clip(-3,3)

def addMa(timeframe):
    p=df['price']
    df['ma'+timeframe] = delta(p,p.rolling(timeframe).mean())

def add2Ma(timeframeshort, timeframelong):
    p=df['price']
    df['ma'+timeframeshort+'_'+timeframelong] =delta(p.rolling(timeframeshort).mean(),p.rolling(timeframelong).mean())


def addMinMax(timeframe,periods,freq):
    p=df['price']
    dfs = df.shift(periods=periods, freq=freq)
    mins = dfs['Low'].rolling(timeframe).min()
    df['maxmin'+timeframe]=((p - mins)/(dfs['High'].rolling(timeframe).max() - mins)).clip(-3,3)
    

def addBol(timeframe):
    p=df['price']
    df['bol'+timeframe] = (delta(p,p.rolling(timeframe).mean())/p.rolling(timeframe).std()).clip(-3,3)

def addStdDir(timeframe,periods,freq):
    p=df['price']
    val=p.rolling(timeframe).std()
    df['stddir'+timeframe] = delta(val,val.shift(periods=periods, freq=freq))

def addMaDir(timeframe,periods,freq):
    p=df['price']
    val = p.rolling(timeframe).mean()
    df['madir'+timeframe] = delta(val,val.shift(periods=periods, freq=freq))







addMa('5min')
addMa('1h')
addMa('5D')
addBol('20D')
addMinMax('10D',1,"D")
addBol('5D')
add2Ma('6min','13min')
add2Ma('6h','13h')
addMaDir('15min',5,"min")
addStdDir('1D',5,"min")

df = df.dropna()

    
del df['High']
del df['Low']

df

,price,ask,bid,volume,ma5min,ma1h,ma5D,bol20D,maxmin10D,bol5D,ma6min_13min,ma6h_13h,madir15min,stddir1D
date,,,,,,,,,,,,,,
2012-01-03 17:26:00,5.29,5.3429,5.2371,11.000000,0.000000,9.541985e-03,0.071139,0.203399,1.475410,0.203399,0.014382,2.220446e-16,0.000000,-0.039384
2012-01-03 17:27:00,5.29,5.3429,5.2371,4.010815,0.000000,7.139457e-03,0.066398,0.190185,1.475410,0.190185,0.000000,2.220446e-16,0.004794,-0.076904
2012-01-04 12:57:00,5.19,5.2419,5.1381,8.724470,0.000000,2.220446e-16,0.042064,0.132066,0.860215,0.132066,0.000000,2.569170e-02,0.000000,-0.050786
2012-01-04 15:53:00,5.32,5.3732,5.2668,0.186090,0.000000,1.236917e-02,0.062841,0.200012,1.000000,0.200012,0.000000,1.902439e-02,0.012524,0.008568
2012-01-04 15:54:00,5.32,5.3732,5.2668,10.394737,0.000000,8.212255e-03,0.059945,0.190974,1.000000,0.190974,0.000000,1.729970e-02,0.025048,0.004977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,-0.000404,3.096598e-04,0.049769,0.000017,0.867563,0.000027,0.000530,-1.043274e-03,0.000471,-0.001104
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,-0.000325,3.125800e-04,0.049747,0.000017,0.867543,0.000027,0.000361,-1.040359e-03,0.000554,-0.001444
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,0.000236,9.555471e-04,0.050408,0.000017,0.871506,0.000027,0.000238,-1.038407e-03,0.000575,-0.001463


In [12]:
from collections import deque
import numpy as np
import pandas as pd

res = deque([np.nan]*5,maxlen=5)
val= None
up=True
factor = 5/100.0
result = {}
for row in df.itertuples(index=True):
    if val == None: 
        val=row.price
    if up:
        if val < row.ask:
            val=row.ask
        elif val*(1-factor) > row.bid:
            res.appendleft(val)
            up=False
            val = row.bid
    else:
        if val > row.bid:
            val=row.bid
        elif val*(1+factor) < row.ask:
            res.appendleft(val)
            up=True
            val = row.ask
    result[row.Index]=[(row.price/res[0])-1,(row.price/res[1])-1,(row.price/res[2])-1,(row.price/res[3])-1,(row.price/res[4])-1]
pd.DataFrame.from_dict(result, orient='index').dropna()

,0,1,2,3,4
2012-01-13 20:34:00,-0.093533,0.042685,-0.075908,0.063931,-0.035730
2012-01-14 00:57:00,-0.123209,0.008549,-0.106161,0.029100,-0.067298
2012-01-14 00:59:00,-0.123209,0.008549,-0.106161,0.029100,-0.067298
2012-01-14 01:08:00,-0.123209,0.008549,-0.106161,0.029100,-0.067298
2012-01-14 01:09:00,-0.136698,-0.006967,-0.119912,0.013267,-0.081647
...,...,...,...,...,...
2021-03-30 23:56:00,0.084109,0.026736,0.176011,0.091872,0.149508
2021-03-30 23:57:00,0.084105,0.026733,0.176008,0.091869,0.149504
2021-03-30 23:58:00,0.084808,0.027398,0.176770,0.092576,0.150249
2021-03-30 23:59:00,0.085487,0.028041,0.177506,0.093260,0.150969
